In [2]:
import yaml
from PIL import Image
with open('pipeline_config.yaml', 'r') as file:
    pipeline_config = yaml.safe_load(file)

In [7]:
url = 'https://img.freepik.com/free-photo/portrait-american-woman_53876-148185.jpg'

In [5]:
system_prompt = '''
Please describe the garment in this image in detail using the following structure:
1. Type of Garment:
2. Sleeve Length:
3. Neckline/Collar Type:
4. Button or Zipper:
5. Pattern/Design:
6. Dominant Colors:
7. Picture/Graphic (if any): If the garment has any pictures or graphics, describe what they are and their colors.
8. Material/Fabric Type:
9. Fit (e.g., slim, regular, oversized):
10. Hemline Length (if applicable):
11. Special Features (e.g., pockets, embroidery, embellishments):
12. Cuffs (if any):
13. Back Design (if different from the front):
14. Occasion/Style (e.g., casual, formal, athletic):
15. Brand (if visible on the garment):

Please format the text description as if you're writing a reference-style title, for example: 
"Long Sleeve Collared Button-Down Gingham Shirt in Red and White, featuring a soft cotton fabric with a relaxed fit. The shirt has buttoned cuffs and a straight hemline, with no additional graphics or logos. Designed for casual wear, the shirt's lightweight material makes it ideal for everyday use. The gingham pattern gives it a classic, timeless look, while the buttons down the front add a touch of sophistication. Perfect for both layering or wearing alone, this versatile piece suits casual and semi-formal occasions alike."
'''


In [ ]:
from pydantic import BaseModel
from typing import Optional

class GarmentAttributes(BaseModel):
    type_of_garment: str
    sleeve_length: str
    neckline_or_collar_type: str
    button_or_zipper: Optional[str]
    pattern_or_design: Optional[str]
    dominant_colors: str
    picture_or_graphic: Optional[str]
    material_or_fabric_type: Optional[str]
    fit: Optional[str]
    hemline_length: Optional[str]
    special_features: Optional[str]
    cuffs: Optional[str]
    occasion_or_style: Optional[str]
    brand: Optional[str]


# Final super class that holds both structured attributes and long description
class GarmentDescription(GarmentAttributes):
    attrebutes: GarmentAttrebutes
    text_description: str

In [ ]:
from PIL import Image
import io
import base64

def encode_image_pil(pil_image):
    buffered = io.BytesIO()
    pil_image.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode("utf-8")

In [18]:
from pydantic import BaseModel
import openai

# Define your Pydantic model for the expected response structure
class ImageDescription(BaseModel):
    description: str

client = OpenAI(
    api_key=api_key,
)

img = Image.open('example/cloth/Long Sleeve Belted Wrap Coat in Dark Green with Stand Collar.jpg')
img = img.resize((256,256))

In [ ]:
base64_image = encode_image_pil(img)

# Make a request to the OpenAI API with structured outputs
response = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Describe this garment:"},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
            ]
        }
    ],
    response_format=GarmentDescription,
)


# Print the structured description
print(image_description.description)